<a href="https://colab.research.google.com/github/changyuhsin1999/duke_chatbot/blob/main/Mistral_7B_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "duke_chatbot" # Enter repo name
git_path = 'https://github.com/changyuhsin1999/duke_chatbot.git'
!git clone "{git_path}"

Cloning into 'duke_chatbot'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 90 (delta 42), reused 16 (delta 7), pack-reused 0
Receiving objects: 100% (90/90), 8.05 MiB | 12.14 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [2]:
!pip install transformers datasets peft accelerate bitsandbytes trl safetensors torch --no-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 208.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 260.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 273.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 297.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 242.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 158.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 147.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 302.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 134.6 MB/s eta 0:00:00
  

In [3]:
import json
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [5]:
#Load accelerate
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [24]:
#Load data and split data into train test
from sklearn.model_selection import train_test_split
import json

def formatting_func(example):
    text = f"### Question: {example['question']}\n### Answer: {example['answer']}"
    return text

def load_qa_pairs(file_path):
    with open(file_path, 'r') as file:
        qa_pairs = json.load(file)
    return qa_pairs

def format_qa_pairs_from_file(file_path):
    qa_pairs = load_qa_pairs(file_path)
    formatted_pairs = [formatting_func(pair) for pair in qa_pairs]
    return formatted_pairs

file_path = 'duke_chatbot/Question_Answer_pairs.json'  # Update this path to your JSON file's location
formatted_qa_pairs = format_qa_pairs_from_file(file_path)

# Splitting the formatted_qa_pairs into training and testing datasets
train_pairs, test_pairs = train_test_split(formatted_qa_pairs, test_size=0.2, random_state=42)

# Optionally, print the sizes of the train and test sets to verify
print(f"Number of training pairs: {len(train_pairs)}")
print(f"Number of testing pairs: {len(test_pairs)}")

# If you want to see a few examples from the training set
for example in train_pairs[:5]:
    print(example, "\n---\n")

Number of training pairs: 652
Number of testing pairs: 163
### Question: What is the admission process like for Duke University's Artificial Intelligence program?
### Answer: The admission process for Duke University's AI program typically includes submitting online applications, transcripts, letters of recommendation, GRE scores (optional), statement of purpose, and TOEFL/IELTS scores for international students. 
---

### Question: How does the Duke University Artificial Intelligence program stay current with emerging trends in AI research?
### Answer: The Duke University Artificial Intelligence program stays current with emerging trends in AI research through faculty research, collaborations, partnerships, publications, conferences, workshops, and professional development opportunities that allow students and researchers to stay informed and engaged with the latest advancements in the field. 
---

### Question: How does Duke University's AI program contribute to the development of AI

In [8]:
#Load the model
base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [27]:
#Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

In [32]:
def generate_and_tokenize_prompts(examples):
    # Tokenize each example individually and return a list of tokenized examples
    tokenized_examples = [tokenizer(example, truncation=True, padding="max_length", max_length=512) for example in examples]
    return tokenized_examples

train_tokenized_examples = generate_and_tokenize_prompts(train_pairs)
test_tokenized_examples = generate_and_tokenize_prompts(test_pairs)

In [35]:
#Set up LoRA
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [37]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [38]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [39]:
model = accelerator.prepare_model(model)

In [43]:
import transformers
from datetime import datetime

project = "duke_chat_finetuned"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_tokenized_examples,
    eval_dataset=test_tokenized_examples,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=500,
        learning_rate=2.5e-5, # Want a small lr for finetuning
        bf16=False,
        fp16=True,
        optim="paged_adamw_8bit",
        logging_steps=25,              # When to start reporting loss
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=25,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=25,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,1.267800,0.872777
50,0.779800,0.822605
75,0.769700,0.784206
100,0.751700,0.785210
125,0.731100,0.752687
150,0.764500,0.741092
175,0.733400,0.734731
200,0.732400,0.731889
225,0.746600,0.716579
250,0.713100,0.717225


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in

TrainOutput(global_step=500, training_loss=0.6793248748779297, metrics={'train_runtime': 4293.9716, 'train_samples_per_second': 0.233, 'train_steps_per_second': 0.116, 'total_flos': 2.2105194233856e+16, 'train_loss': 0.6793248748779297, 'epoch': 1.53})